Importieren, benötigter Packages und einlesen der Trainings- und Testdaten

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

datensatz = pd.read_csv("./foot-traffic-wue/train.csv")



Trainingsdaten nach Fehlern und unrelevanten Daten bereinigen

In [ ]:
datensatz_f = datensatz[
    ~datensatz['incidents'].isin(['laser_failure']) 
    ]

datensatz_f.reset_index(inplace=True)

Markierung der Feiertage im Datensatz

In [ ]:
# Liste der Feiertage
feiertage = [
    '2019-04-19', '2019-04-22', '2019-05-01', '2019-05-30', '2019-06-10', '2019-06-20', '2019-08-15',
    '2019-10-03', '2019-11-01', '2019-12-25', '2019-12-26', '2020-01-01', '2020-01-06', '2020-04-10',
    '2020-04-13', '2020-05-01', '2020-05-21', '2020-06-01', '2020-06-11', '2020-08-15', '2020-10-03',
    '2020-11-01', '2020-12-25', '2020-12-26', '2021-01-01', '2021-01-06', '2021-04-02', '2021-04-05',
    '2021-05-01', '2021-05-13', '2021-05-24', '2021-06-03', '2021-08-15', '2021-10-03', '2021-11-01',
    '2021-12-25', '2021-12-26', '2022-01-01', '2022-01-06', '2022-04-15', '2022-04-18', '2022-05-01',
    '2022-05-26', '2022-06-06', '2022-06-16', '2022-08-15', '2022-10-03', '2022-11-01', '2022-12-25',
    '2022-12-26', '2023-01-01', '2023-01-06', '2023-04-07', '2023-04-10', '2023-05-01', '2023-05-18',
    '2023-05-29', '2023-06-08', '2023-08-15', '2023-10-03', '2023-11-01', '2023-12-25', '2023-12-26',
    '2024-01-01', '2024-01-06', '2024-03-29', '2024-04-01', '2024-05-01', '2024-05-09', '2024-05-20',
    '2024-05-30', '2024-08-15'
]

# Konvertiere die Feiertage zu einem Set für eine schnellere Suche
feiertage_set = set(feiertage)

# Erstelle die neue Spalte "is_feiertag"
datensatz_f['is_feiertag'] = datensatz_f['date'].apply(lambda x: 1 if x in feiertage_set else 0)


Aufteilung der Spalte date in year, month und day 

In [ ]:
# Konvertiere die Spalte 'date' in ein Datetime-Format
datensatz_f['date'] = pd.to_datetime(datensatz_f['date'])

# Zerlege die Spalte in 'year', 'month' und 'day'
datensatz_f['year'] = datensatz_f['date'].dt.year
datensatz_f['month'] = datensatz_f['date'].dt.month
datensatz_f['day'] = datensatz_f['date'].dt.day


Zusammenführen der Wetterbedingungen, welche in Tag und Nacht unterteilt sind 

In [ ]:
# Werte zusammenführen
datensatz_f['weather_condition'] = datensatz_f['weather_condition'].replace({
    'partly-cloudy-day': 'partly-cloudy',
    'partly-cloudy-night': 'partly-cloudy',
    'clear-day': 'clear',
    'clear-night': 'clear'
})

Aufstellen des Modells mit Random Forest

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

# Daten laden (Hier muss der korrekte DataFrame 'datensatz_f' bereits existieren)
feature_columns = ["streetname", "hour", "weekday", "weather_condition", "incidents", "temperature", "year", "month", "day"]
target_columns = ["n_pedestrians", "n_pedestrians_towards", "n_pedestrians_away"]

X = datensatz_f[feature_columns]
Y = datensatz_f[target_columns]

# Kategorische Spalten encodieren
categorical_columns = ["streetname", "weekday", "weather_condition", "incidents"]
encoder = OrdinalEncoder()
X_categorical_encoded = encoder.fit_transform(X[categorical_columns])
X_categorical_encoded_df = pd.DataFrame(X_categorical_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Numerische Features hinzufügen
X_encoded = pd.concat([X_categorical_encoded_df, X[["hour", "temperature", "year", "month", "day"]].reset_index(drop=True)], axis=1)

# Sicherstellen, dass X_train und Y_train synchron sind
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size=0.1, random_state=42)

# 10-fache Cross validation
kf = KFold(n_splits=10, shuffle=True)

# Hyperparameter-Grid
param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [10, 30, 50],
    'max_features': [0.3, 0.5, 1.0],
}


# Grid Search für Random Forest
rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=32, verbose=0)
grid_search.fit(X_encoded, Y)

# Beste Parameter ausgeben
print("Beste Parameterwerte:")
for param, value in grid_search.best_params_.items():
    print(f"{param}: {value}")

# Bestes Modell trainieren
best_params = grid_search.best_params_
best_rf_model = RandomForestRegressor(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], max_features=best_params['max_features'], random_state=42)
best_rf_model.fit(X_train, Y_train)

# Vorhersagen
best_predictions = best_rf_model.predict(X_test)

# Fehlerberechnung
mse = mean_squared_error(Y_test, best_predictions)
rmse = np.sqrt(mse)
print(f"Beste MSE: {mse}")
print(f"Beste RMSE: {rmse}")
